# Save a script to train a classification model

The purpose is to test Azure MLOps tools with a classification problem. 

My usual test case is to classify daily particle size distributions (that was what I did in Uno when doing my PhD, so know and like the problem). In short, there are days when something happens (event days), days when something does not happen (non-event days) and mixed days (undefined days). There we were try to build a model to predict days into these three. 

There exists have concentration data and classification data. Both have mixed quality, some data missing and some wrong. The classification data is also provided with higher granularity than described above: there is class 1a, 1b, and 2 events. Here we however aggregate them all. 

This notebook creates a training script that uses Tensorflow with Keras frontend to build up a CNN classifier. The classifier is simple and bad, but it is fast to run. :) 

Data from training process and test data confusion matrix are saved in Azure Machine Learning workspace to keep track on the performance of the model. 





### Create a folder experiment files

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = '../classification_scripts'
os.makedirs(experiment_folder, exist_ok=True)
#print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/aerosol_training.py

# Import libraries
import os
import argparse
import joblib
import time

from azureml.core import Run

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
from tensorflow.keras import datasets, layers, models
import tensorflow as tf


# parameters

# how to scale logaritmic data
data_scaling={"use_limits":True,
             'min':-3, 
             'max':5}
log_offset=1e-3
classes=3


###
# Get the experiment run context
run = Run.get_context()
run.log('Info1','script started ok')

###
#load the diabetes data (passed as an input dataset)
print("Loading Data...")

hyde = run.input_datasets['hyde_data'].to_pandas_dataframe()
hyde_class=run.input_datasets['class_data'].to_pandas_dataframe()

os.makedirs('outputs', exist_ok=True)
hyde.sample(10).to_csv("outputs/sample_hydedata.csv", index=False, header=True)
hyde_class.sample(10).to_csv("outputs/sample_classdata.csv", index=False, header=True)

### Data cleaning
# Get date in datetimeindex format, drop original columns

hyde['date']=pd.to_datetime(hyde[['Year', 'Month', 'Day', 'Hour','Minute','Second']])
hyde=hyde.drop(['Year', 'Month', 'Day', 'Hour','Minute','Second'], axis=1)
hyde.set_index('date', inplace=True)

hyde_class['date']=pd.to_datetime(hyde_class['date'])
hyde_class.set_index('date', inplace=True)

# convert columnnames to shorter, they show the particle size in nm
dps=[(item,item[10:]) for item in hyde.columns if item.startswith('HYY_DMPS.d')]
dps_2={name:np.round((float(item[:3])/1000* 10**(int(item[-1]))),2) for (name, item) in dps}
cols_dict=dict(zip(hyde.columns, dps_2))
hyde=hyde.rename(columns=dps_2)

# drop metadata columns off
metacolumns=[ 'HYY_DMPS.t1',    'HYY_DMPS.t2',    'HYY_DMPS.p1',    'HYY_DMPS.p2',
         'HYY_DMPS.rh1',   'HYY_DMPS.rh2', 'HYY_DMPS.tconc',]
hydemeta=hyde[metacolumns]
hyde=hyde.drop(metacolumns, axis=1)

run.log_list('hyde data columns', list(hyde.columns))

###

# data cleaning: replace negative values with zeros, 
# replace nan's using a linear interpolation (this is a time serie) 

hyde[hyde<0]=0
hyde=hyde.fillna( method ='ffill', axis=0)
hyde=hyde.fillna(0)

###

# Merge classification data and the measurement data so that they have same days

data_days=hyde.groupby([hyde.index.date])
data_days=pd.to_datetime(list(data_days.groups))

class_days=hyde_class.index

common_days=[item.date() for item in sorted(list(set(data_days) & set(class_days)))]


# create hyde_crop dataset so that all days have a classification
hyde_crop=hyde.reset_index()
hyde_crop=hyde_crop[hyde_crop['date'].dt.date.isin(common_days)]
hyde_crop=hyde_crop.set_index('date')


# crop classification data so all days have aerosol data
class_crop=hyde_class.loc[common_days]


# test if indeces are ok
assert set(hyde_crop.index.date)^set(class_crop.index)==set()

###

# Log-transform and scale the measurement data to better suit for the ML algorithm
# this could be changed to use scalers from scikit learn
# but now it is in line with the traditional scaling

log_offset=1e-6
hyde_log=(hyde_crop+log_offset).apply(np.log10)

mini, maxi=hyde_log.min().min() , hyde_log.max().max()

if data_scaling['use_limits']:
    hyde_norm=(hyde_log-0.5*(maxi-10+mini))/(maxi-2-mini)
else:
    hyde_norm=(hyde_log-(data_scaling['min']+data_scaling['max'])/2)/(data_scaling['max']+data_scaling['min'])

###

# Create day-based arrays that are in right format for Keras model
# data_s if not normed data
# data_n is normed data

# create an array of arrays for each day containing particle data
#data_s = np.array([  grp.values.T for key, grp in hyde_crop.groupby([hyde_crop.index.date]) if len(grp.values)==144] )
data_n = np.array([  grp.values.T for key, grp in hyde_norm.groupby([hyde_norm.index.date]) if len(grp.values)==144] )
# array of dates
dates_s=np.array([  key for key, grp in hyde_crop.groupby([hyde_crop.index.date]) if len(grp.values)==144] )

#print('The size of the data matrices: '+ str(data_s.shape) + ' ,' + str(data_n.shape))


###

# Make arrays from the classification data for the same dates as the hyde_data is
# Three arrays, for three different classification schemes

print(hyde_class.head())

# only the relevant columns
classes_s=hyde_class[['event Ia','event Ib','event II','non event','undefined']]
classes_5s=classes_s[classes_s.index.isin(dates_s)]

# combine all event columns to one column, build up a new df
only_events=pd.DataFrame(classes_5s[['event Ia', 'event Ib', 'event II']].sum(axis=1))
only_events.columns=['event']
others=classes_5s[[ 'undefined','non event']]
classes_3s=pd.merge(only_events, others, left_index=True, right_index=True)

# combine all 1 class event columns to one column, build up a new df
only_events=pd.DataFrame(classes_5s[['event Ia', 'event Ib']].sum(axis=1))
only_events.columns=['event I']
others=classes_5s[[ 'event II', 'undefined','non event']]
classes_4s=pd.merge(only_events, others, left_index=True, right_index=True)

# COnvert to numpy arrays
classes_3=np.array(classes_3s)
classes_4=np.array(classes_4s)
classes_5=np.array(classes_5s)

#print(classes_3.shape)
#print(data_s.shape)


###

# Split to train and test sets
X = data_n
X = X.reshape( X.shape+(1,)  )

if classes==3:
    y=classes_3
elif classes==4:
    y=classes_4
elif classes==5:
    y=classes_5
    
#print(X.shape)
#print(y.shape)
#print(classes)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)


###
# build CNN model

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=( 61,144,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))


model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(3,activation='softmax'))

model.summary()


###

t1=time.time()

model.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x=X_train, y=y_train, epochs=10,
                    callbacks=[callback],
                    validation_data=(X_test, y_test))

t2=time.time()

run.log('model train time (secs)',round (t2 - t1))

print(("It takes %s seconds to train the model.") % round (t2 - t1))


###


from sklearn.metrics import confusion_matrix
import itertools

if classes==3:
    class_names=['Event', 'Undefined','Non-event']
elif classes==4:
    class_names=['Event I','Event II', 'Undefined','Non-event']
elif classes==5:
    class_names=['Event Ia', 'Event Ib', 'Event II', 'Non-event', 'Undef']

# yt: real classification for the test data
# pr: predicted classification for the test data
yt=y_test.argmax(axis=1)
pr=np.argmax(model.predict(X_test), axis=1)

print(yt[:5])
print(pr[:5])
print(len(pr), len(yt))

# Compute confusion matrix and tranform it to right format for log_table function
cnf_matrix = confusion_matrix(yt, pr)

cf_dict={}
for row in range(cnf_matrix.shape[0]):
    cf_dict[row]=list(cnf_matrix[row])
    
run.log_table('confusion_matrix', cf_dict)

def plot_confusion_matrix(f, ax, cm, class_names,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    #This function prints and plots the confusion matrix.
    #Normalization can be applied by setting `normalize=True`.
    
    ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.set_title(title)
    #plt.colorbar()
    tick_marks = np.arange(len(class_names))
    ax.set_xticks(tick_marks)
    ax.set_xticklabels(class_names, rotation=45)
    ax.set_yticks(tick_marks)
    ax.set_yticklabels(class_names)

    ax.set_xlim([-0.5,len(class_names)-0.5])
    ax.set_ylim([-0.5,len(class_names)-0.5])
    ax.invert_yaxis()

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        ax.text(j, i, np.round(cm[i, j],2),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')
    f.tight_layout()

    
    return f

# Plot non-normalized confusion matrix
f, ax=plt.subplots()
plot_confusion_matrix(f, ax, cnf_matrix, class_names=class_names,\
                      title='Confusion matrix, without normalization')

run.log_image('confusion matrix',plot=f)

# Save the trained model in the outputs folder

model.save('outputs/aerosol_classification_model.h5')
#joblib.dump(value=model, filename='outputs/aerosol_classification_model.pkl')

run.complete()
